In [1]:
import tweepy
from kafka import KafkaProducer
from json import dumps
import time
import json

### Authentication on Twitter

In [2]:
consumer_key = "dmn51aowr3bYzIWz3ELgARN5x"
consumer_secret = "Q6vHf7FNj1yDvX2vggN6kKLgiDP7t7HL5DNpNvK0akyjzfxTFt"
access_token = "164266780-hcTZEl441WUst50TpiRIZtAd3IHmvPDX5tQUDjqx"
access_token_secret = "AIoEwy9AL52AuHRAKesBph3arofQ4AdLwqYRl8tFnehxd"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

### Connection to Apache Kafka

In [4]:
#Create Kafka Producer

#ip = 'localhost'
ip = '35.228.250.247'
producer = KafkaProducer(bootstrap_servers=[ip+':9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [16]:
#Retrieve TopN Hashtags on the given location

location = 2487956 #san francisco
n = 5 #top n parameter
top_n = [x.get("name") for x in api.trends_place(location)[0].get('trends')[0:n]]
print(top_n)

['Niners', '#MUNLIV', '#RaiderNation', 'Liverpool', 'Jimmy G']


### Pipelined Producer

In [ ]:
#Define Listener

class PipelineListener(tweepy.StreamListener):
    def __init__(self,tag,time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        self.tag=tag
        super(PipelineListener, self).__init__()

    def on_status(self, status):
        if (time.time() - self.start_time) < self.limit:
            data = {}
            data['tag'] = self.tag
            data['date'] = str(status.created_at).split(" ")[0]#Keep the date, remove time
            data['tweet'] = status.text
            producer.send("DIC",data)
            print(data)
            return True
        else:
            print("Stopping stream..")
            return False

#Launch listener in series

time_limit = 100

for _ in range(0,5):
    listeners = []
    streams = []
    for i in range(0,n):
        print(top_n[i])
        listeners.append(PipelineListener(top_n[i],time_limit=time_limit))
        streams.append(tweepy.Stream(auth = api.auth, listener=listeners[i]))
        streams[i].filter(track=[top_n[i]], is_async=False, languages=["en"])
        streams[i].disconnect()

Niners
{'tag': 'Niners', 'date': '2019-10-20', 'tweet': 'Would it be good or bad for the Panthers next week if the Skins somehow stayed competive with the niners today....'}
{'tag': 'Niners', 'date': '2019-10-20', 'tweet': 'ADD: Bills 2H -9.5(-105)'}
{'tag': 'Niners', 'date': '2019-10-20', 'tweet': 'Exactly!'}
{'tag': 'Niners', 'date': '2019-10-20', 'tweet': '@AlSacco49 Depends. Are you a diehard Faithful?  If so then I’d say nope. Should be watching Niners'}
{'tag': 'Niners', 'date': '2019-10-20', 'tweet': 'I missed my bet for the niners. Don’t look like a bad miss now. Let’s get this W tho!! #go49ers #SFvsWAS'}
{'tag': 'Niners', 'date': '2019-10-20', 'tweet': 'RT @E40: Niners mane! @RSherman_25 @49ers let’s go! https://t.co/0QsqWFDAHn'}
{'tag': 'Niners', 'date': '2019-10-20', 'tweet': 'RT @E40: Niners mane! @RSherman_25 @49ers let’s go! https://t.co/0QsqWFDAHn'}
{'tag': 'Niners', 'date': '2019-10-20', 'tweet': 'RT @49erHodge: Good thing the Niners paid Robbie Gould. 👎 He needs to get

{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': "RT @ManUtd: Today's #MUFC Man of the Match, with 47 per cent of the votes: @Daniel_James_97 🙌\n\n#MUNLIV https://t.co/04dqreZsWW"}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'RT @premierleague: "We need to be better but we need to take it as it is. We’ll take the point – it’s much better than no points" - Jurgen…'}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'RT @NathEbruba: Leicester 2nd since getting rid of Maguire, Swansea 3rd since getting rid of Daniel James, Palace 6th since getting rid of…'}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': '90 min has sayed it all'}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'RT @premierleague: "We need to be better but we need to take it as it is. We’ll take the point – it’s much better than no points" - Jurgen…'}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'RT @AdityaBamane: ~ Not Giving 3 consecutive years to the managers\n~ High Wages,low value of money\n~ Players 

{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'RT @Mbaka_Ogonnaya: Rashford strips Van Dijk of the imaginary Balon Dor Liverpool fans gave him. 😂😂😂\n#MUNLIV https://t.co/DCSJXCvJY8'}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': "RT @ManUtd: Today's #MUFC Man of the Match, with 47 per cent of the votes: @Daniel_James_97 🙌\n\n#MUNLIV https://t.co/04dqreZsWW"}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': "The worst feelings is knowing I can't go back and fix what I've ruined, help me O'Lord #Liverpool #MUNLIV #Rashford #lecntertainment"}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'RT @farouq_yahaya: Rashford has his 49th Utd goal at the expense of a fuming German 😂 😂 : \n\nPeter Drury talking about Liverpool coach disag…'}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'Deserved the honour! #MUFC'}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'RT @PlayBoi_Young44: Rare video of Van Dijk searching for greener pastures 😹😹 #MUNLIV https://t.co/yRf0IB2PQA'}
{'tag'

{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': "RT @ManUtd: Today's #MUFC Man of the Match, with 47 per cent of the votes: @Daniel_James_97 🙌\n\n#MUNLIV https://t.co/04dqreZsWW"}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'RT @devil0697: You will always be special❤️ 🎶Jose Mourinho🎶 #MUNLIV https://t.co/qdKMcBci89'}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'RT @Sxbnkz: Like and rt if you can see your team #MUNLIV 😂😂😂 https://t.co/zWPgf6YoNB'}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': "RT @bet_clever: Gary Neville's reaction Rashford's opening goal. #MUNLIV [Sky] https://t.co/udzJLFUvIB"}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'RT @Leeroy2F2S: If you’re gonna complain about a ‘foul’ in the lead up to the goal, at least hold the correct leg that’s been tapped. Muppe…'}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': "RT @__mrjoe: In my lifetime of watching football I've never seen Manchester united this bad and Liverpool this good. #MUNLIV"}
{'tag':

{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'RT @ManUtd: That NOISE from Old Trafford when we took the lead! 😍\n\n#MUFC #MUNLIV https://t.co/jtp7oIV8uH'}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': "RT @poshsypce22: Ike: *stirs d pap vigorously.. says don't say i didn't try. U can go shower nw pls go enjoy ur shower .\n\nMe:😂 *Ike is litt…"}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'RT @The_Kalebae: Liverpool so instead of doing practice you were playing basketball? See what that cost you #MUNLIV'}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': "RT @jymsQto: #MUNLIV Liverpool fun thinking they can win the league when they can't beat even Manchester United https://t.co/XGBRFzMCQA"}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'RT @farouq_yahaya: Klopp did enough to keep the unbeaten run going...and just enough for Ole to keep his job. Absolute mind games #MUNLIV'}
{'tag': '#MUNLIV', 'date': '2019-10-20', 'tweet': 'RT @ManUtd: The points are shared after a mu

...

{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'RT @Carra23: What time am I doing this?!! https://t.co/wltQhIRX3C'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': "RT @jeremycorbyn: We're getting ready for the fight of our lives.\n\nhttps://t.co/NqxWNjwDwT"}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'RT @TrollFootball: Liverpool dropped points against Man Utd 😂😭😂 https://t.co/F5XNslB45M'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': "RT @TrollFootball: A minute silence for every Liverpool supporter who thought they would win the league when they can't even beat Man Utd."}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'Liverpool would’ve won that game if Keita started, he’s ready'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'It was nice seeing Marcus Rashford scoring against Liverpool, again, and I was wrong about Harry Maguire - he’s a f… https://t.co/4R5XTqY6wU'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'We have 16 Conclusions from 

{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': "RT @GoalNews: 'Even the best things have to come to an end, sometime.'\n\n@neiljonesgoal reports from Old Trafford, as Lallana rescues below-…"}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': "RT @utdrobbo: Dear Jurgen Klopp &amp; Liverpool,\n\n• Don't dive on minimal contact.\n• Don't use your hands to control the ball.\n\nThen you don't…"}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'RT @AnfieldWatch: 📺 Jordan Henderson says Liverpool could have played better in their 1-1 draw with Manchester United while Adam Lallana ag…'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'RT @ThoseScouseLads: Don’t think I’ll ever get over Manchester United playing 5 at the back at home to Liverpool. Iconic'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'RT @SuperbiaProeIia: 🔞 Longest winning streaks in top flight league football in England:\n\n1️⃣8️⃣ Manchester City\n1️⃣7️⃣ Liverpool\n\n#ManCity…'}
{'tag': 'Liverpool', 'dat

{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'Utd fans hoping for defeat v Liverpool to remove OGS ?#BehindClosedDoors'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'RT @TrollFootball: Liverpool dropped points against Man Utd 😂😭😂 https://t.co/F5XNslB45M'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'RT @PrimeOxlade: @TierneysWay Any of these:\n- 6 stars in their name\n- Something related to European champions in their bio\n- A cringe locat…'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': "RT @TheMasterBucks: When my girlfriend tells me she's a Liverpool fan https://t.co/sC70UCoXw8"}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'Liverpool not their best at all today but got to look at the positives, still unbeaten and still top but a long lon… https://t.co/p8MXR2FLx3'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'RT @AndyMitten: Loads of inflatable 6s in the Liverpool end - to represent the number of times they’ve nearly won the league since 1

{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': '@LeyeAkinola Yea, it was just a matter of Liverpool literally not turning up'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'RT @piersmorgan: United’s goal came after a clear foul on Origi. Liverpool’s disallowed goal came after a clearly accidental use of an arm…'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'RT @rioferdy5: Unsettled Liverpool, made it difficult for them. Seeing out games like that when put under pressure is where you earn ya mon…'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'Liverpool drops first points in 1-1 draw at Man United : Liverpool drops first points in 1-1 draw at Man United… https://t.co/txwrGgXORL'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': 'RT @volqx: I have always known Liverpool to be a goal scoring team even against big teams like City x Arsenal...\nIt’s unfortunate they can’…'}
{'tag': 'Liverpool', 'date': '2019-10-20', 'tweet': '@macncheese065 @ManUtd Mr Hairy If 

{'tag': 'Jimmy G', 'date': '2019-10-20', 'tweet': 'JIMMY G IS TRASH'}
{'tag': 'Jimmy G', 'date': '2019-10-20', 'tweet': '@roheblius shorty G &gt;&gt; jimmy G'}
{'tag': 'Jimmy G', 'date': '2019-10-20', 'tweet': 'I love Jimmy but this a stupid ass tweet Young is an all time great'}
{'tag': 'Jimmy G', 'date': '2019-10-20', 'tweet': 'Jimmy G isn’t good'}
Stopping stream..
Niners
{'tag': 'Niners', 'date': '2019-10-20', 'tweet': 'Unsurprising as both are basically from the same WCO tree where you zone run to set up the passing game.'}
{'tag': 'Niners', 'date': '2019-10-20', 'tweet': '@ProfessorBZ Haha. I thought you were a niners fan?'}
{'tag': 'Niners', 'date': '2019-10-20', 'tweet': '@Shrube3 You been criticizing the niners all season unprovoked at that but telling  me not to be a hater stop it rn'}
{'tag': 'Niners', 'date': '2019-10-20', 'tweet': 'Niners tied with the Skins????'}
{'tag': 'Niners', 'date': '2019-10-20', 'tweet': 'Let’s get it niners!!!! #GoNiners'}
{'tag': 'Niners', 'date'

### Parallel Producer

In [ ]:
#Define Listener

class ParallelListener(tweepy.StreamListener):
    def __init__(self,time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        super(ParallelListener, self).__init__()

    def on_status(self, status):
        if (time.time() - self.start_time) < self.limit:
            data = {}
            data['date'] = str(status.created_at)
            data['tweet'] = status.text
            json_data = json.dumps(data)
            producer.send("test",json_data)
            print(json_data)
            return True
        else:
            print("Stopping stream..")
            return False

#Launch all listener together

listener = ParallelListener(time_limit=1000)
myStream = tweepy.Stream(auth = api.auth, listener=listener)
myStream.filter(track=top_n, is_async=False, languages=["en"])